Links:
- https://github.com/hardmaru/WorldModelsExperiments
- http://blog.otoro.net//2018/06/09/world-models-experiments/


In [2]:
%matplotlib inline
import numpy as np
import importlib
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import gym
import torch
import torch.nn as nn
from PIL import Image
import PIL
from torchsummary import summary

In [3]:
env = gym.make('MountainCar-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


Mountain Car Environment

In [4]:
state = env.reset()
ims = []
for i in range(1000):
    action = env.action_space.sample()
    state, r, done, _ = env.step(action)
    im = env.render(mode='rgb_array')
    ims.append(im)
env.close()

In [7]:
resized_ims = [Image.fromarray(i).resize((64, 64), Image.BILINEAR) for i in ims]

In [17]:
resized_ims = [np.array(i) for i in resized_ims]

In [18]:
resized_ims = np.array(resized_ims)

In [ ]:
torch.distributions.distribution.Distribution()

In [73]:
help(torch.normal)

Help on built-in function normal:

normal(...)
    .. function:: normal(mean, std, out=None) -> Tensor
    
    Returns a tensor of random numbers drawn from separate normal distributions
    whose mean and standard deviation are given.
    
    The :attr:`mean` is a tensor with the mean of
    each output element's normal distribution
    
    The :attr:`std` is a tensor with the standard deviation of
    each output element's normal distribution
    
    The shapes of :attr:`mean` and :attr:`std` don't need to match, but the
    total number of elements in each tensor need to be the same.
    
    .. note:: When the shapes do not match, the shape of :attr:`mean`
              is used as the shape for the returned output tensor
    
    Args:
        mean (Tensor): the tensor of per-element means
        std (Tensor): the tensor of per-element standard deviations
        out (Tensor, optional): the output tensor
    
    Example::
    
        >>> torch.normal(mean=torch.arange(1., 11

In [67]:
class VAE(nn.Module):
    def __init__(self, z_size=32, batch_size=1, learning_rate=0.0001, kl_tolerance=0.5, gpu=False):
        self.z_size=z_size,
        self.batch_size=batch_size, 
        self.learning_rate=learning_rate, 
        self.kl_tolerance=kl_tolerance
        self.gpu = gpu
        
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2),
            nn.ReLU(),
        )
        
        self.mu = nn.Linear(256*2*2, self.z_size)
        self.logvar = nn.Linear(256*2*2, self.z_size)
        self.sigma = torch.exp(self.logvar/2.0)
        self.eps = torch.normal()
        
    def forward(self, x):
        out = self.encoder(x)
        
        mu = self.mu(out)
        sigma = 
        return out
# h = tf.layers.conv2d(self.x, 32, 4, strides=2, activation=tf.nn.relu, name="enc_conv1")
# h = tf.layers.conv2d(h, 64, 4, strides=2, activation=tf.nn.relu, name="enc_conv2")
# h = tf.layers.conv2d(h, 128, 4, strides=2, activation=tf.nn.relu, name="enc_conv3")
# h = tf.layers.conv2d(h, 256, 4, strides=2, activation=tf.nn.relu, name="enc_conv4")
# h = tf.reshape(h, [-1, 2*2*256])
    
# self.mu = tf.layers.dense(h, self.z_size, name="enc_fc_mu")
#       self.logvar = tf.layers.dense(h, self.z_size, name="enc_fc_log_var")
#       self.sigma = tf.exp(self.logvar / 2.0)
#       self.epsilon = tf.random_normal([self.batch_size, self.z_size])
#       self.z = self.mu + self.sigma * self.epsilon


In [68]:
v = VAE()

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = VAE().to(device)

In [70]:
model

VAE(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2))
    (5): ReLU()
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2))
    (7): ReLU()
  )
)

In [71]:
var = torch.from_numpy(resized_ims)
var = var.reshape((1000, 3, 64, 64)).float()

In [61]:
x = model.forward(var)

In [62]:
x.shape

torch.Size([1000, 32, 31, 31])

In [72]:
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 31, 31]           1,568
              ReLU-2           [-1, 32, 31, 31]               0
            Conv2d-3           [-1, 64, 14, 14]          32,832
              ReLU-4           [-1, 64, 14, 14]               0
            Conv2d-5            [-1, 128, 6, 6]         131,200
              ReLU-6            [-1, 128, 6, 6]               0
            Conv2d-7            [-1, 256, 2, 2]         524,544
              ReLU-8            [-1, 256, 2, 2]               0
Total params: 690,144
Trainable params: 690,144
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 0.75
Params size (MB): 2.63
Estimated Total Size (MB): 3.43
----------------------------------------------------------------
